In [2]:
import sys
sys.path.append("V:/biomoni/") 
from biomoni import Experiment
from BacillusScripts.BacillusVariableFeedrate_copy import Bacillus_vf
from biomoni import Model
import pandas as pd
import numpy as np

In [8]:
a = Experiment("V:/biomoni/BacillusData/Stamm185", "F5", endpoint="F_end")

b= Bacillus_vf()


In [17]:
a.dataset["off"]["Glucose [g/L]"].iloc[5]

34.162697

In [ ]:
def calc_CRR(self,experiment,c, y0):

    Mw_bm = 24.445          #g/mol molecular weight of biomass assuming composition of C=1,H=1.594,N=0.293,O=0.387,P=0.012,S=0.005
    Mw_gluc = 180.156       #g/mol molecular weight of glucose C=6,H=12,O=6
    Mw_RF = 376.36          #g/mol molecular weight of Riboflavin C=17,H=20,N=4,O=6
    F_C_in = c["feedrate_glc"](t_grid)
        
    amountC_given = (y0[0]/Mw_bm)+(y0[1]/Mw_gluc *6)+(y0[2]/Mw_RF *17)

    t_grid = experiment.dataset["off"].index.values

    for t in t_grid:
        Carb_RR=[]
        nC_S = experiment.dataset["off"].loc[t,"Glucose [g/L]"]
        nC_X = experiment.dataset["off"].loc[t,"Glucose [g/L]"]
        amountC_found = nC_S + nC_X + nC_CO2_cum
        Carb_RR = amountC_found / amountC_given
        
        return Carb_RR